<a href="https://colab.research.google.com/github/mohanrajmit/AgeCalculation/blob/master/CNN_AgeEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Age Estimation for Indian Movie Face Dataset like Young, Old and Middle

In [1]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

fatal: destination path 'AgeCalculation' already exists and is not an empty directory.


In [6]:
# keras imports

from keras.applications.xception import Xception, preprocess_input
from keras.applications import xception
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import pandas as pd




In [7]:
data=pd.read_csv("AgeCalculation/dataset/train.csv")



In [8]:
data.head()

,ID,Class
0,377.jpg,MIDDLE
1,17814.jpg,YOUNG
2,21283.jpg,MIDDLE
3,16496.jpg,YOUNG
4,4487.jpg,MIDDLE


In [9]:
data_test=pd.read_csv("AgeCalculation/dataset/test.csv")

In [10]:
data_test.head()

,ID
0,25321.jpg
1,989.jpg
2,19277.jpg
3,13093.jpg
4,5367.jpg


In [11]:
print(data.head())

          ID   Class
0    377.jpg  MIDDLE
1  17814.jpg   YOUNG
2  21283.jpg  MIDDLE
3  16496.jpg   YOUNG
4   4487.jpg  MIDDLE


In [12]:
image_name = data["ID"]
label_name = data["Class"]

features = []
labels = []

In [13]:
test_image_name=data_test["ID"]

In [14]:
features_test=[]

In [15]:
dataset_path="AgeCalculation/dataset/Train/"

In [16]:
test_dataset_path="AgeCalculation/dataset/Test/"

In [17]:
for image_name1,label in zip(image_name,label_name):

	print("the image name  {} and label{}" .format(image_name1,label))
	image_path= str(dataset_path+image_name1)
	#print(image_path)
	img =cv2.imread(image_path,0)
	x = cv2.resize(img, (48, 48),interpolation=cv2.INTER_AREA)
	#images.append(image)
	#labels.append(label)

	#img = image.load_img(image_path, target_size=(73, 73))
	#x = image.img_to_array(img)
	features.append(x)
	labels.append(label)

Streaming output truncated to the last 5000 lines.
the image name  11430.jpg and labelMIDDLE
the image name  14976.jpg and labelYOUNG
the image name  8698.jpg and labelMIDDLE
the image name  7492.jpg and labelMIDDLE
the image name  6134.jpg and labelMIDDLE
the image name  13508.jpg and labelMIDDLE
the image name  421.jpg and labelMIDDLE
the image name  8340.jpg and labelOLD
the image name  20983.jpg and labelMIDDLE
the image name  1284.jpg and labelMIDDLE
the image name  15799.jpg and labelMIDDLE
the image name  15439.jpg and labelMIDDLE
the image name  25435.jpg and labelMIDDLE
the image name  9079.jpg and labelMIDDLE
the image name  14915.jpg and labelMIDDLE
the image name  10023.jpg and labelMIDDLE
the image name  675.jpg and labelMIDDLE
the image name  21824.jpg and labelMIDDLE
the image name  21605.jpg and labelYOUNG
the image name  21594.jpg and labelYOUNG
the image name  20541.jpg and labelMIDDLE
the image name  7368.jpg and labelMIDDLE
the image name  3301.jpg and labelYOUNG
th

In [18]:
for image_name1 in test_image_name:

	print("the image name  {} and label{}" .format(image_name1,label))
	image_path= str(test_dataset_path+image_name1)
	#print(image_path)
	img =cv2.imread(image_path,0)
	x = cv2.resize(img, (48, 48),interpolation=cv2.INTER_AREA)
	#images.append(image)
	#labels.append(label)

	#img = image.load_img(image_path, target_size=(73, 73))
	#x = image.img_to_array(img)
	features_test.append(x)


Streaming output truncated to the last 5000 lines.
the image name  259.jpg and labelMIDDLE
the image name  6144.jpg and labelMIDDLE
the image name  1256.jpg and labelMIDDLE
the image name  26061.jpg and labelMIDDLE
the image name  4326.jpg and labelMIDDLE
the image name  20522.jpg and labelMIDDLE
the image name  5048.jpg and labelMIDDLE
the image name  24562.jpg and labelMIDDLE
the image name  24462.jpg and labelMIDDLE
the image name  20075.jpg and labelMIDDLE
the image name  15719.jpg and labelMIDDLE
the image name  9648.jpg and labelMIDDLE
the image name  7217.jpg and labelMIDDLE
the image name  6742.jpg and labelMIDDLE
the image name  11660.jpg and labelMIDDLE
the image name  6402.jpg and labelMIDDLE
the image name  563.jpg and labelMIDDLE
the image name  15973.jpg and labelMIDDLE
the image name  15580.jpg and labelMIDDLE
the image name  4145.jpg and labelMIDDLE
the image name  18068.jpg and labelMIDDLE
the image name  12173.jpg and labelMIDDLE
the image name  19468.jpg and labelMID

In [19]:
# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)




In [20]:
le_labels

array([0, 2, 0, ..., 0, 0, 0])

In [21]:
from keras.utils import to_categorical

In [22]:
list(le.inverse_transform([0, 1, 2]))

['MIDDLE', 'OLD', 'YOUNG']

In [23]:
num_classes=3

In [24]:
y_train = to_categorical(le_labels, num_classes)

In [25]:
y_train[10]

array([0., 0., 1.], dtype=float32)

In [26]:
features=np.array(features)



In [27]:
features_test=np.array(features_test)



In [28]:
features.shape

(19906, 48, 48)

In [29]:
features_test.shape

(6636, 48, 48)

In [30]:
x_train = features.astype('float32')
x_train /= 255


In [31]:
x_test = features_test.astype('float32')
x_test /= 255


In [32]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [33]:
from keras.layers import Input
from keras.models import Model
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(48,48,1),padding='same', activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(3, activation="softmax"))

In [34]:
from __future__ import print_function

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                   

In [36]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [37]:
model.fit(x_train,y_train,
          batch_size=32,
          epochs=10)

Epoch 1/10
623/623 [==============================] - 11s 8ms/step - loss: 0.8382 - accuracy: 0.6093
Epoch 2/10
623/623 [==============================] - 5s 8ms/step - loss: 0.7444 - accuracy: 0.6672
Epoch 3/10
623/623 [==============================] - 5s 8ms/step - loss: 0.6527 - accuracy: 0.7188
Epoch 4/10
623/623 [==============================] - 5s 8ms/step - loss: 0.5740 - accuracy: 0.7636
Epoch 5/10
623/623 [==============================] - 5s 9ms/step - loss: 0.4633 - accuracy: 0.8098
Epoch 6/10
623/623 [==============================] - 6s 9ms/step - loss: 0.3348 - accuracy: 0.8696
Epoch 7/10
623/623 [==============================] - 5s 9ms/step - loss: 0.2108 - accuracy: 0.9205
Epoch 8/10
623/623 [==============================] - 5s 8ms/step - loss: 0.1227 - accuracy: 0.9546
Epoch 9/10
623/623 [==============================] - 5s 8ms/step - loss: 0.0810 - accuracy: 0.9709
Epoch 10/10
623/623 [==============================] - 5s 9ms/step - loss: 0.0502 - accuracy: 0.983

In [38]:
predictions = model.predict(x_test)

208/208 [==============================] - 1s 3ms/step


In [39]:
model.save("age_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [40]:
# save model to json
model_json = model.to_json()
with open("age_model.json", "w") as json_file:
    json_file.write(model_json)

In [43]:
predictions[0]

array([9.9999988e-01, 9.8665780e-13, 1.0844386e-07], dtype=float32)

In [44]:
np.argmax(predictions[0])

0